<a href="https://colab.research.google.com/github/nangchung/truyvandaphuong/blob/main/%C4%91%E1%BB%99_%C4%91o_TF_IDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob
import numpy as np
import re

In [3]:
def load_data_in_a_directory(data_path):
    file_paths = glob.glob(data_path)
    lst_contents = []
    for file_path in file_paths:
        f = open(file_path, 'r',encoding="utf-8")
        str = f.read()
        words = str.replace('"', '').replace('.', '').replace("'","").split()
        #words = set(words)
        lst_contents.append(words)
    return (lst_contents, file_paths)

In [4]:
# Doc noi dung cua tung file txt
# va xay dung tap "dictionary" chua danh sach cac tu
def build_dictionary(contents):
    dictionary = set()
    for content in contents:
        dictionary.update(content)
    return dictionary

In [5]:
def calc_tf_weighting_inverted(vocab, contents):
    TFinverted = []
    
    for indexW, word in enumerate(vocab):
        files = []
        
        for indexT, title in enumerate(contents):
            file = []
            if word in title:
                file.append(indexT)
                file.append(title.count(word) / len(title))
                files.append(file)
#                 print(file)
        TFinverted.append(files) 

    return TFinverted

In [6]:
def calc_tf_weighting_inverted_query(vocab, contents, querys):
    TFinverted = []
    set_querys = set(querys)
    
        
    for query in set_querys:
        if (query not in vocab):
            continue;
        else:
            for indexW, word in enumerate(vocab):
                file = []
                if (query == word):
                    file.append(indexW)
                    file.append(querys.count(query) / len(querys))

            #                 print(file)
                    TFinverted.append(file) 

    return TFinverted

In [7]:
# MAIN

# BUOC 1: Load cac file trong 'data' va xay dụng tap cac tu vung
contents, paths = load_data_in_a_directory('/content/drive/My Drive/truy van da phuong/filedata/*.txt')
vocab = build_dictionary(contents)

In [8]:
# BUOC 2: Xay dung vector TF weighting cho 
# tap van ban va truy van
TF = calc_tf_weighting_inverted(vocab, contents)
query="kỹ sư "
qcontent = query.split()
qTF = calc_tf_weighting_inverted_query(vocab, contents, qcontent)

In [9]:
# BUOC 3: Xay dung vector IDF weight cho tap van ban
DF = np.array([len(x) for x in TF])
IDF = 1 + np.log(len(contents) / DF)
IDF = np.array([IDF]).T
IDF

array([[6.81114099],
       [6.11799381],
       [7.90975328],
       ...,
       [2.60644837],
       [6.81114099],
       [7.90975328]])

In [10]:
qTF

[[1365, 0.5], [4599, 0.5]]

In [11]:
# BUOC 4: Xay dung vector TF_IDF weighting cho
# tap van ban va truy van
for i in range(len(IDF)):
    for j in range(len(TF[i])):
#         print(TF2[i])
#         print("\n")
        TF[i][j][1] = TF[i][j][1] * IDF[i]
for q in qTF:
    q[1] *= IDF[q[0]]

In [12]:
TF

[[[178, array([0.00858908])],
  [197, array([0.00649918])],
  [453, array([0.02904538])]],
 [[207, array([0.00527868])],
  [315, array([0.00891836])],
  [555, array([0.00935473])],
  [596, array([0.00424566])],
  [696, array([0.0094268])],
  [889, array([0.00856862])]],
 [[237, array([0.01120362])]],
 [[292, array([0.00909365])],
  [453, array([0.05809075])],
  [919, array([0.00644384])]],
 [[968, array([0.01233971])]],
 [[424, array([0.03584811])],
  [606, array([0.02536738])],
  [907, array([0.00949288])]],
 [[141, array([0.00233004])],
  [294, array([0.00746664])],
  [326, array([0.00893042])],
  [417, array([0.00476518])],
  [448, array([0.01422726])],
  [567, array([0.0077462])],
  [659, array([0.01345614])],
  [675, array([0.00697376])],
  [677, array([0.00760385])],
  [749, array([0.00755756])],
  [778, array([0.00721703])],
  [806, array([0.00241152])],
  [891, array([0.00484421])],
  [892, array([0.00502054])],
  [945, array([0.00490159])],
  [956, array([0.00168888])],
  [959

In [13]:
# BUOC 5: Tinh do tuong dong cua query va cac van ban
# su dung TF_IDF weighting

#Độ đo L2
dists = np.zeros(len(contents))

for q in qTF:
    for tf_idf in TF[q[0]]:
        dists[tf_idf[0]] += ((q[1] - tf_idf[1]))**2
        
# BUOC 6: Sap xep de sap hang va hien thi ket qua
rank = np.argsort(dists)[::-1]
print(rank)
print(dists)
topK = 1
for i in range(topK):
    print('Văn bản gần thứ ', i+1, ' theo độ đo L2 là: ', ' '.join(paths[rank[i]]))

[195 531  44 ... 628 627   0]
[0. 0. 0. ... 0. 0. 0.]
Văn bản gần thứ  1  theo độ đo L2 là:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 6 4 1 . t x t


In [14]:
import numpy as np

In [15]:
d_query = 0
for q in qTF:
  d_query += q[1]**2
d_query = np.sqrt(d_query)
d_query

array([2.09907996])

In [16]:
#Độ đo cosine
dis_doc = np.zeros(len(contents))
cosines = np.zeros(len(contents))
dot_qd = np.zeros(len(contents))

for q in qTF:
  for tf_idf in TF[q[0]]:
    dis_doc[tf_idf[0]] += tf_idf[1]**2
    dot_qd[tf_idf[0]] += tf_idf[1]*q[1]

for i in range (len(dis_doc)):
  if dis_doc[i]==0:
    continue
  else:
    cosines[i]+=dot_qd[i]/d_query*np.sqrt(dis_doc[i])
cosines

array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
rank1 = np.argsort(cosines)[::-1]
rank1

array([207, 436, 571, ..., 628, 627,   0])

In [18]:
top=1
for i in range(top):
    print('Văn bản gần thứ ', i+1, 'theo độ đo cosine là: ', ' '.join(paths[rank1[i]]))

Văn bản gần thứ  1 theo độ đo cosine là:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 6 5 3 . t x t


In [22]:
#Tích vô hướng
scalar1 = np.zeros(len(contents))

for q in qTF:
  for tf_idf in TF[q[0]]:
    scalar1[tf_idf[0]] += tf_idf[1]*q[1]


In [23]:
scalar1

array([0., 0., 0., ..., 0., 0., 0.])

In [25]:
rank2 = np.argsort(scalar1)[::-1]
rank2

array([207, 436, 571, ..., 628, 627,   0])

In [26]:
top2=1
for i in range(top2):
    print('Văn bản gần thứ ', i+1, ' theo độ đo tích vô hướng là: ', ' '.join(paths[rank2[i]]))

Văn bản gần thứ  1  theo độ đo tích vô hướng là:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 6 5 3 . t x t


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
